<a href="https://colab.research.google.com/github/ferragina/MyInformationRetrieval/blob/main/6_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LLM in practice**

In [3]:
!pip install transformers>=4.41.2 accelerate>=0.31.0


In [4]:
# import the required classes
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cpu",
    torch_dtype="auto",
    trust_remote_code=True,
)

# Generating a Text Response to a Prompt

You'll now use the pipeline object (labeled as generator) to generate a response consisting of 50 tokens to the given prompt.

In [6]:
prompt = "The capital of France is"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

In [14]:
model_output = model.model(input_ids)

model_output[0].shape

# The first number represents the batch size, which is 1 in this case since we have one prompt.
# The second number 5 represents the number of tokens.
# And finally 3072 represents the embedding size (the size of the vector that corresponds to each token).

torch.Size([1, 5, 3072])

In [15]:
# The transformer block outputs for each token a vector of size 3072 (embedding size). Let's check the shape of this output.

lm_head_output = model.lm_head(model_output[0])

lm_head_output.shape

# The LM head outputs for each token in the input prompt, a vector of size 32064 (vocabulary size).
# So there are 5 vectors, each of size 32064.
# Each vector can be mapped to a probability distribution, that shows the probability for each token in the vocabulary to come after the given token in the input prompt.

torch.Size([1, 5, 32064])

Since we're interested in generating the output token that comes after the last token in the input prompt ("is"), we'll focus on the last vector. So in the next cell, lm_head_output[0,-1] is a vector of size 32064 from which you can generate the token that comes after ("is"). You can do that by finding the id of the token that corresponds to the highest value in the vector lm_head_output[0,-1] (using argmax(-1), -1 means across the last axis here).

In [12]:
token_id = lm_head_output[0,-1].argmax(-1)

tokenizer.decode(token_id)

'Paris'